In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
from fundl.layers.normalizing_flow import K_planar_flows
from fundl.weights import add_K_planar_flow_params
import autograd.numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Initialize a normal distribution

# Adjust planar flow param initializations

What if we started with planar flow params that were initialized way outside of normal(0, 1)?

In [10]:
from autograd.numpy.random import normal
def add_planar_flow_params(params, name, dim, p):
    """
    Copied from fundl
    """
    params[name] = dict()
    params[name]["w"] = normal(size=(dim, 1), **p)
    params[name]["b"] = normal(**p)
    params[name]["u"] = normal(size=(dim, 1), **p)
    return params


def add_K_planar_flow_params(params, K, dim, p):
    """
    Copied from fundl
    """
    Ks = []
    for i in range(K):
        name = f"planar_flow_{i}"
        params = add_planar_flow_params(params, name, dim, p)
        Ks.append(name)
    return params, Ks

In [11]:
from ipywidgets import interact, FloatSlider, IntSlider

In [12]:
a = np.random.normal(size=(1000, 1))

@interact(
    loc=FloatSlider(min=-10, max=10), 
    scale=FloatSlider(min=0.001, max=20),
    K=IntSlider(min=1, max=10)
)
def plot_original_transformed(loc, scale, K):
    p = dict(loc=loc, scale=scale)
    params = dict()
    params, K_names = add_K_planar_flow_params(params, K=K, dim=1, p=p)
    z_tfm, ldj = K_planar_flows(params, a, K_names)
    sns.kdeplot(*z_tfm.T, label='transformed')
    sns.kdeplot(*a.T, label='original')

interactive(children=(FloatSlider(value=0.0, description='loc', max=10.0, min=-10.0), FloatSlider(value=0.001,…

In [20]:
a2 = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0],[0, 1]], size=1000)


@interact(
    loc=FloatSlider(min=-10, max=10), 
    scale=FloatSlider(min=0.001, max=20),
    K=IntSlider(min=1, max=10)
)
def plot_multivariate_nf(loc, scale, K):
    p = dict(loc=loc, scale=scale)

    params = dict()
    params, K_names = add_K_planar_flow_params(params, K=K, dim=2, p=p)

    z_tfm, ldj = K_planar_flows(params, a2, K_names)
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), sharex=True, sharey=True)
    axes[1].scatter(*z_tfm.T, alpha=0.1)
    axes[1].set_title('transformed')

    axes[0].scatter(*a2.T, alpha=0.1)
    axes[0].set_title('original')

interactive(children=(FloatSlider(value=0.0, description='loc', max=10.0, min=-10.0), FloatSlider(value=0.001,…